In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
from ultralytics import YOLO
from ultralytics.yolo.v8.detect.predict import DetectionPredictor
from ultralytics.yolo.utils.plotting import Annotator
import os
import supervision as sv

/home/anas/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = YOLO("/mnt/F68C0A428C09FE3D/Projects/minesweepers material/localization/yl8/best(1).pt")

In [3]:
# img = os.path.join('Data', "image_0.png")

In [4]:
# results=model.predict(source="4",show = True)
# print(results)

In [5]:
def calculate_distance(focal_length, object_width, pixel_width):
    distance = (focal_length * object_width) / pixel_width
    return distance

def focal_length_finder (measured_distance, real_width, width_in_rf):
    focal_length = (width_in_rf * measured_distance) / real_width

    return focal_length


# focal_length_finder(30,20,w)

In [6]:

def main():
    
    # box_annotator = sv.BoxAnnotator(thickness=2, text_thickness=2, text_scale=1)
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        results=model.predict(frame,show = True)

        for r in results:
            annotator = Annotator(frame)
            boxes = r.boxes
            for box in boxes:
                b = box.xyxy[0]  # get box coordinates in (top, left, bottom, right) format
                c = box.cls
                w = box.xywh[0][2]
                h = box.xywh[0][3]
                
                # fl = focal_length_finder(30,20,w)
                # print("focal length = ",fl)

                distance = calculate_distance(800, 20, w)
                # print("distance = ",distance)
                # convert the distance from tensor to float
                


                # #get the width and height of the box
                # print("width = ", b[2] - b[0])
                # print("height =", b[3] - b[1])
                # print("width = ", box.xywh[0][2])
                # print("height = ", box.xywh[0][3])
                # print(c) #class
                # print(model.names[int(c)]) #class name
                # print(box) #box object
                # print(box.xyxy[0]) #box coordinates
                # print(box.conf) #confidence

                annotator.box_label(b, model.names[int(c)] + " " + str(distance.item()))
#                 annotator.box_label(b, model.names[int(c)])

        frame = annotator.result()  

        cv2.imshow('YOLO', frame)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break 
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


0: 480x640 (no detections), 6.8ms
Speed: 0.4ms preprocess, 6.8ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 6.8ms
Speed: 0.6ms preprocess, 6.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 7.8ms
Speed: 0.9ms preprocess, 7.8ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 6.3ms
Speed: 0.5ms preprocess, 6.3ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 6.5ms
Speed: 0.5ms preprocess, 6.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 5.9ms
Speed: 0.4ms preprocess, 5.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 8.8ms
Speed: 0.5ms preprocess, 8.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.2ms
Speed: 0.8ms preprocess, 12.2ms inference, 0.6m


0: 480x640 1 circle, 13.3ms
Speed: 0.6ms preprocess, 13.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.5ms
Speed: 0.6ms preprocess, 13.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.8ms
Speed: 0.7ms preprocess, 12.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.4ms
Speed: 0.6ms preprocess, 13.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.1ms
Speed: 0.6ms preprocess, 12.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.6ms
Speed: 0.6ms preprocess, 12.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 9.9ms
Speed: 0.5ms preprocess, 9.9ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.3ms
Speed: 0.7ms preprocess, 13.3ms inference,


0: 480x640 (no detections), 13.3ms
Speed: 0.7ms preprocess, 13.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.8ms
Speed: 0.7ms preprocess, 12.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.7ms
Speed: 0.8ms preprocess, 12.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 14.0ms
Speed: 0.6ms preprocess, 14.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 15.0ms
Speed: 0.6ms preprocess, 15.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 13.4ms
Speed: 0.6ms preprocess, 13.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 13.2ms
Speed: 0.6ms preprocess, 13.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 12.7ms
Speed: 0.6ms preprocess, 12.7ms inference, 1.1ms postprocess 

Speed: 0.4ms preprocess, 4.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 9.5ms
Speed: 0.6ms preprocess, 9.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 6.2ms
Speed: 0.4ms preprocess, 6.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 6.1ms
Speed: 0.5ms preprocess, 6.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 7.2ms
Speed: 0.4ms preprocess, 7.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 5.7ms
Speed: 0.4ms preprocess, 5.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 13.6ms
Speed: 0.6ms preprocess, 13.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 14.3ms
Speed: 0.7ms preprocess, 14.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 12.9ms
Spe


0: 480x640 (no detections), 13.0ms
Speed: 0.6ms preprocess, 13.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 14.2ms
Speed: 0.6ms preprocess, 14.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.1ms
Speed: 0.7ms preprocess, 12.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.1ms
Speed: 0.6ms preprocess, 13.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.2ms
Speed: 0.6ms preprocess, 13.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.4ms
Speed: 0.7ms preprocess, 13.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.3ms
Speed: 0.8ms preprocess, 13.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.2ms
Speed: 0.8ms preprocess, 13.2ms i

...


0: 480x640 (no detections), 12.6ms
Speed: 0.6ms preprocess, 12.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.9ms
Speed: 0.6ms preprocess, 13.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.1ms
Speed: 0.6ms preprocess, 13.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.2ms
Speed: 0.6ms preprocess, 13.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.9ms
Speed: 0.6ms preprocess, 12.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.4ms
Speed: 0.6ms preprocess, 13.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.8ms
Speed: 0.6ms preprocess, 12.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.3ms
Speed: 0.6ms preprocess, 13.3ms i


0: 480x640 (no detections), 12.4ms
Speed: 0.8ms preprocess, 12.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.1ms
Speed: 0.7ms preprocess, 12.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 14.2ms
Speed: 0.8ms preprocess, 14.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.2ms
Speed: 0.7ms preprocess, 12.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.5ms
Speed: 0.8ms preprocess, 12.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.0ms
Speed: 0.7ms preprocess, 13.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.9ms
Speed: 0.7ms preprocess, 12.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.9ms
Speed: 0.8ms preprocess, 12.9ms i


0: 480x640 (no detections), 13.3ms
Speed: 0.8ms preprocess, 13.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.3ms
Speed: 0.8ms preprocess, 13.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.1ms
Speed: 0.8ms preprocess, 13.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.3ms
Speed: 0.8ms preprocess, 12.3ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.0ms
Speed: 0.7ms preprocess, 13.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.6ms
Speed: 0.7ms preprocess, 12.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 12.7ms
Speed: 0.7ms preprocess, 12.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.1ms
Speed: 0.8ms preprocess, 13.1ms i


0: 480x640 (no detections), 13.0ms
Speed: 0.6ms preprocess, 13.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.3ms
Speed: 0.7ms preprocess, 13.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 13.0ms
Speed: 0.6ms preprocess, 13.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 13.5ms
Speed: 0.7ms preprocess, 13.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 13.8ms
Speed: 0.6ms preprocess, 13.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 2 circles, 12.9ms
Speed: 0.7ms preprocess, 12.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 12.9ms
Speed: 0.6ms preprocess, 12.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 13.3ms
Speed: 0.6ms preprocess, 13.3ms inference, 1.2ms postprocess per im


0: 480x640 1 circle, 12.5ms
Speed: 0.6ms preprocess, 12.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 12.8ms
Speed: 0.6ms preprocess, 12.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 12.1ms
Speed: 0.6ms preprocess, 12.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 12.7ms
Speed: 0.6ms preprocess, 12.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 12.4ms
Speed: 0.7ms preprocess, 12.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 13.7ms
Speed: 0.6ms preprocess, 13.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 12.0ms
Speed: 0.6ms preprocess, 12.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 circle, 12.2ms
Speed: 0.6ms preprocess, 12.2ms inference, 1.2ms postprocess per image at shape (1, 3, 64

In [ ]:
# results.names()